<a href="https://colab.research.google.com/github/gwangjinjeong/Box-office-prediction-model/blob/master/Daily_audience_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Crawling the Box office data Without KOFIC API 

In [1]:
from google.colab import drive
drive.mount('./gdrive')
%cd ./gdrive/My Drive/BoxOffice

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive
/content/gdrive/My Drive/BoxOffice


In [0]:
import time, re, requests,sys
from bs4 import BeautifulSoup
from datetime import datetime

`calendar.monthrange(year, month)`는 tuple을 return하고, 이 tuple의

첫 번째 원소는 month의 1일이 무슨 요일인지   
두 번째 원소는 month가 총 며칠인지 를 나타냅니다.

In [0]:
import pandas as pd
from bs4 import BeautifulSoup
class Daily_boxoffice():
  """
  # Crawling the box office data in Korea Film Council class
  # Author : Gwangjin jeong
  # Date : 20.03.22
  # Description
  ## input 
  ### year, month, day, mode
  ### mode = 1 : 전체를 다 불러온다.
  ### mode = 2 : top 10만 불러온다.
  ## output
  ###processed dataframe  
  """
  def __init__(self, year,month,day,mode):
    self.year = year
    self.month = month
    self.day = day
    self.mode = mode
    self.base_url = "http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom="
    self.all_data = []
    now = datetime.now()

    if (now.year == year) & (now.month == month) & (now.day == day):
      return 'error'
    elif now.year == year:
      self._mon_range = range(1,now.month)
    else:
      self._mon_range = range(1,13)

  def cleanText(self,text):
    remove_space = re.sub(re.compile(r'\s+'), '', text)
    remove_word1 = re.sub('[동일,New]','',remove_space)
    remove_word2 = re.sub('([0-9]+)상승','',remove_word1)
    remove_word3 = re.sub('([0-9]+)하락','',remove_word2)
    return remove_word3

  def get_movie_data(self):
    tag_body_top10 = self.soup.find('tbody', attrs={'id': 'tbody_0'})
    tag_body_else = self.soup.findAll('tbody')
    if self.mode == 1: # Top 110
      for tbody in tag_body_else:
        for value in tbody.findAll('td'):
          self.all_data.append(self.cleanText(value.text))
    elif self.mode == 2: # Top 10
      for value in tag_body_top10.findAll('td'):
        self.all_data.append(self.cleanText(value.text))

  def slicing_list(self,lst,n):
    divide = int(len(lst)/n)
    return [lst[start:start+divide] for start in range(0,len(lst),divide)]

  def getWebPage(self):
    url = self.base_url+str(self.year)+"-" + str(self.month).zfill(2)+"-" + str(self.day).zfill(2) + "&sSearchTo="+str(self.year)+"-"+str(self.month).zfill(2)+"-" + str(self.day).zfill(2)    
    source_code = requests.get(url)
    plain_text = source_code.text
    self.soup = BeautifulSoup(plain_text, 'html.parser' )
    date = str(self.year)+"-" + str(self.month).zfill(2)+"-" + str(self.day).zfill(2)
    # daily
    print(f'Downloading boxoffice at {date}....') # 일자
    # movie ranking
    self.get_movie_data()
    if self.mode == 1:
      return (date, self.slicing_list(self.all_data,110))
    elif self.mode == 2:
      return (date, self.slicing_list(self.all_data,10))

In [0]:
import pandas as pd
from tqdm.notebook import tqdm
import calendar
def get_daily_boxOffice_for_a_year(year,mode):
  result_columns = ['조회날자','순위','영화명','개봉일','매출액','매출액점유율','매출액증감(전일대비)','누적매출액','관객수','관객수증감(전일대비)','누적관객수','스크린수','상영횟수']
  result = pd.DataFrame(columns=result_columns)
  now = datetime.now()

  for month in tqdm(range(1,13)):
      for day in range(1,calendar.monthrange(year,month)[1]+1):
          if (now.year == year) & (now.month == month) & (now.day == day):
              result.to_csv(f'{year}_boxoffice.csv')
              return result
          temp = Daily_boxoffice(year,month,day,mode)
          crawling_data = temp.getWebPage()
          date = crawling_data[0]
          boxoffice = crawling_data[1]
          for day in boxoffice:
            result = result.append(pd.DataFrame([[date]+day], columns=result_columns), ignore_index=True)
  result.to_csv(f'{year}_boxoffice.csv')
  return result

In [17]:
for year in range(2012,2020+1):
  %%time
  get_daily_boxOffice_for_a_year(year,2)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.68 µs



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


AttributeError: ignored

In [20]:
get_daily_boxOffice_for_a_year(2020,2)

,조회날자,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),누적매출액,관객수,관객수증감(전일대비),누적관객수,스크린수,상영횟수
0,2020-01-01,1,백두산,2019-12-19,4430968920,37.3%,1900850950(75.1%),57336776690,498802,182606(57.8%),6789306,1236,5845
1,2020-01-01,2,천문:하늘에묻는다,2019-12-26,2092505020,17.6%,916870400(78.0%),9900747970,239149,89345(59.6%),1204773,929,3439
2,2020-01-01,3,시,2019-12-18,1691272350,14.3%,589055650(53.4%),22927750150,190455,54187(39.8%),2716441,801,2997
3,2020-01-01,4,미드웨이,2019-12-31,1561075860,13.2%,489446340(45.7%),2646735380,174407,42527(32.2%),307780,810,2614
4,2020-01-01,5,겨울왕국2,2019-11-21,727968600,6.1%,288393600(65.6%),112324217320,83529,28067(50.6%),13452593,665,1459
...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,2020-03-21,6,지푸라기라도잡고싶은짐승들,2020-02-19,23902020,4.0%,10450180(77.7%),5287711470,2662,1162(77.5%),609198,294,524
806,2020-03-21,7,세인트아가타,2020-03-19,24238480,4.0%,5532640(29.6%),68713240,2648,636(31.6%),7728,370,685
807,2020-03-21,8,스타이즈본,2018-10-09,12643880,2.1%,3998540(46.3%),4391308300,2478,783(46.2%),517094,211,313
808,2020-03-21,9,어바웃타임,2013-12-05,7810500,1.3%,3799500(94.7%),25217042300,1522,736(93.6%),3434190,81,112
